In [23]:
#Loading Packages
import pandas as pd

In [24]:
#cloning github
!git clone https://github.com/kirthi-b/QoL_NYC_Children

fatal: destination path 'QoL_NYC_Children' already exists and is not an empty directory.


In [25]:
#loading file
df_econ = pd.read_excel("QoL_NYC_Children/Economic/Data/Copy of econ_2019_acs5yr_cdta.xlsx")

In [5]:
df_econ.head()

,GeoType,CDTAType,GeogName,GeoID,Borough,Pop16plE,Pop16plM,Pop16plC,Pop16plP,Pop16plZ,LFE,LFM,LFC,LFP,LFZ,CvLF1E,CvLF1M,CvLF1C,CvLF1P,CvLF1Z,CvEm16pl1E,CvEm16pl1M,CvEm16pl1C,CvEm16pl1P,CvEm16pl1Z,CvLFUEm1E,CvLFUEm1M,CvLFUEm1C,CvLFUEm1P,CvLFUEm1Z,LFArmdFE,LFArmdFM,LFArmdFC,LFArmdFP,LFArmdFZ,NLF1E,NLF1M,NLF1C,NLF1P,NLF1Z,...,Pv125t149E,Pv125t149M,Pv125t149C,Pv125t149P,Pv125t149Z,Pv150t174E,Pv150t174M,Pv150t174C,Pv150t174P,Pv150t174Z,Pv175t184E,Pv175t184M,Pv175t184C,Pv175t184P,Pv175t184Z,Pv185t199E,Pv185t199M,Pv185t199C,Pv185t199P,Pv185t199Z,Pv200t299E,Pv200t299M,Pv200t299C,Pv200t299P,Pv200t299Z,Pv300t399E,Pv300t399M,Pv300t399C,Pv300t399P,Pv300t399Z,Pv400t499E,Pv400t499M,Pv400t499C,Pv400t499P,Pv400t499Z,Pv500plE,Pv500plM,Pv500plC,Pv500plP,Pv500plZ
0,CDTA2020,CD,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),BK01,Brooklyn,149365,2148.0,0.9,100.0,0.0,104042,1969.0,1.2,69.7,0.9,104015,1971.0,1.2,69.6,0.9,98706,1885.0,1.2,66.1,0.8,5309,667.0,7.6,3.6,0.4,27,44.0,99.1,0.0,0.0,45323,1487.0,2.0,30.3,0.9,...,7904,865.0,6.7,4.2,0.5,6105,680.0,6.8,3.2,0.4,2254,427.0,11.5,1.2,0.2,3772,592.0,9.5,2.0,0.3,22120,1340.0,3.7,11.7,0.7,15880,1084.0,4.2,8.4,0.6,15770,1078.0,4.2,8.3,0.6,56672,1740.0,1.9,29.9,0.8
1,CDTA2020,CD,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,BK02,Brooklyn,99920,1688.0,1.0,100.0,0.0,71505,1481.0,1.3,71.6,0.9,71495,1481.0,1.3,71.6,0.9,67057,1401.0,1.3,67.1,0.8,4438,506.0,6.9,4.4,0.5,10,15.0,91.2,0.0,0.0,28415,1087.0,2.3,28.4,1.0,...,2220,394.0,10.8,2.0,0.3,3289,464.0,8.6,2.9,0.4,863,217.0,15.3,0.8,0.2,1333,274.0,12.5,1.2,0.2,9658,734.0,4.6,8.5,0.6,7455,698.0,5.7,6.6,0.6,8135,634.0,4.7,7.2,0.5,60404,1623.0,1.6,53.3,1.1
2,CDTA2020,CD,BK03 Bedford-Stuyvesant (CD 3 Approximation),BK03,Brooklyn,134612,2329.0,1.1,100.0,0.0,89218,1958.0,1.3,66.3,0.9,89164,1958.0,1.3,66.2,0.9,82739,1880.0,1.4,61.5,0.9,6425,618.0,5.9,4.8,0.5,54,45.0,51.1,0.0,0.0,45394,1564.0,2.1,33.7,1.0,...,7534,731.0,5.9,4.5,0.4,7598,726.0,5.8,4.5,0.4,2223,347.0,9.5,1.3,0.2,4399,546.0,7.5,2.6,0.3,22953,1228.0,3.3,13.7,0.7,17653,1042.0,3.6,10.5,0.6,15765,1000.0,3.9,9.4,0.6,39873,1484.0,2.3,23.8,0.8
3,CDTA2020,CD,BK04 Bushwick (CD 4 Equivalent),BK04,Brooklyn,92766,2023.0,1.3,100.0,0.0,63722,1792.0,1.7,68.7,1.2,63722,1792.0,1.7,68.7,1.2,60146,1748.0,1.8,64.8,1.2,3576,436.0,7.4,3.9,0.5,0,NaN,NaN,NaN,NaN,29044,1203.0,2.5,31.3,1.1,...,4707,603.0,7.8,4.2,0.5,6372,663.0,6.3,5.7,0.6,2141,428.0,12.2,1.9,0.4,3867,557.0,8.8,3.4,0.5,18722,1177.0,3.8,16.7,1.0,14608,990.0,4.1,13.0,0.8,9587,774.0,4.9,8.5,0.7,17987,1006.0,3.4,16.0,0.8
4,CDTA2020,CD,BK05 East New York-Cypress Hills (CD 5 Approxi...,BK05,Brooklyn,146181,2474.0,1.0,100.0,0.0,82374,2002.0,1.5,56.4,1.0,82317,2002.0,1.5,56.3,1.0,76337,1923.0,1.5,52.2,1.0,5980,566.0,5.8,4.1,0.4,57,54.0,58.1,0.0,0.0,63807,1814.0,1.7,43.6,1.0,...,10663,899.0,5.1,5.7,0.5,8296,767.0,5.6,4.4,0.4,4221,614.0,8.8,2.2,0.3,6180,674.0,6.6,3.3,0.4,35436,1558.0,2.7,18.8,0.8,20053,1164.0,3.5,10.7,0.6,16766,1117.0,4.0,8.9,0.6,25155,1298.0,3.1,13.4,0.6


In [26]:
#filtering for important columns, here median income, splitting code and name, and create new dataframe
df_eco = pd.DataFrame(data={"Code" : list(df_econ["GeogName"].str.split(" ", 1, expand = True).reset_index()[0]),
                            "CD_Name" : list(df_econ["GeogName"].str.split(" ", 1, expand = True).reset_index()[1]),
                            "Median_Income" : list(df_econ['MdHHIncE'])
                           }
                     )

In [ ]:
df_eco

,Code,CD_Name,Median_Income
0,BK01,Williamsburg-Greenpoint (CD 1 Equivalent),73595.0
1,BK02,Downtown Brooklyn-Fort Greene (CD 2 Approximat...,103475.0
2,BK03,Bedford-Stuyvesant (CD 3 Approximation),56440.0
3,BK04,Bushwick (CD 4 Equivalent),55385.0
4,BK05,East New York-Cypress Hills (CD 5 Approximation),39162.0
...,...,...,...
66,QN84,Jamaica Bay (East) (JIA 84 Approximation),49998.0
67,SI01,North Shore (CD 1 Equivalent),68126.0
68,SI02,Mid-Island (CD 2 Approximation),86817.0
69,SI03,South Shore (CD 3 Approximation),96785.0


In [7]:
df_eco.shape[0]

71

In [27]:
df_eco1 = pd.DataFrame(data={"Code" : list(df_econ["GeogName"].str.split(" ", 1, expand = True).reset_index()[0]),
                            "Median_Income" : list(df_econ['MdHHIncE'])
                           }
                     )

df_eco1

,Code,Median_Income
0,BK01,73595.0
1,BK02,103475.0
2,BK03,56440.0
3,BK04,55385.0
4,BK05,39162.0
...,...,...
66,QN84,49998.0
67,SI01,68126.0
68,SI02,86817.0
69,SI03,96785.0


In [ ]:
df_eco1.to_csv('MedIncome1.csv', index = False)

In [ ]:
from google.colab import files
files.download("MedIncome1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Spatial Join with Table Attributes? 

In [9]:
#import dataset for equivalencies between PUMA Codes and Community District Codes
df_puma = pd.read_excel("QoL_NYC_Children/Economic/Data/PUMA_CD.xlsx")
df_puma

,Borough,County Code,Borough Code,Census Tract,PUMA,Code,Name
0,Bronx,5,2,31000,3704,BX31,Allerton-Pelham Gardens
1,Bronx,5,2,31200,3704,BX31,Allerton-Pelham Gardens
2,Bronx,5,2,31400,3704,BX31,Allerton-Pelham Gardens
3,Bronx,5,2,31600,3704,BX31,Allerton-Pelham Gardens
4,Bronx,5,2,31800,3704,BX31,Allerton-Pelham Gardens
...,...,...,...,...,...,...,...
2163,Staten Island,85,5,16901,3903,SI07,Westerleigh
2164,Staten Island,85,5,18701,3903,SI07,Westerleigh
2165,Staten Island,85,5,18901,3903,SI07,Westerleigh
2166,Staten Island,85,5,19700,3903,SI07,Westerleigh


In [10]:
df_puma.dtypes

Borough          object
County Code       int64
Borough Code      int64
Census Tract      int64
PUMA              int64
Code             object
Name             object
dtype: object

In [ ]:
sorted(df_puma['Code'].unique())
sorted(df_eco1['Code'].unique())

In [ ]:
df_puma['PUMA'] = df_puma['PUMA'].astype(str)
df_puma['Code'] = df_puma['Code'].astype(str)
df_puma.dtypes


Borough          object
County Code       int64
Borough Code      int64
Census Tract      int64
PUMA             object
Code             object
Name             object
dtype: object

In [ ]:
#filter for relevant information
df_puma = df_puma[['PUMA','Code']]
df_puma.head()

,PUMA,Code
0,3704,BX31
1,3704,BX31
2,3704,BX31
3,3704,BX31
4,3704,BX31


In [ ]:
#drop duplicates and null values
df_puma = df_puma.drop_duplicates(subset=['Code'])
df_puma = df_puma.reset_index(drop=True)


In [ ]:
df_puma

,PUMA,Code
0,3704,BX31
1,3706,BX05
2,3705,BX06
3,3704,BX07
4,3705,BX01
...,...,...
190,3901,SI32
191,3903,SI37
192,3902,SI24
193,3903,SI22


In [ ]:
#table merge with economics data with PUMA based on "Code"
df_medincome = pd.merge(df_puma, df_eco1, on='Code', how='left')
df_medincome = df_medincome.dropna().reset_index(drop=True)
df_medincome


,PUMA,Code,Median_Income
0,3706,BX05,30217.0
1,3705,BX06,25548.0
2,3704,BX07,37973.0
3,3705,BX01,25741.0
4,3702,BX03,27563.0
5,3710,BX27,54998.0
6,3703,BX10,59353.0
7,3709,BX09,41641.0
8,3701,BX28,14998.0
9,3709,BX08,61182.0


In [ ]:
df_medincome = df_medincome.dropna()

In [ ]:
df_medincome.shape[0]

29

In [ ]:
###Lizzie stopped here


In [ ]:
df_eco.to_csv('MedIncome.csv', index = False)

In [ ]:
from google.colab import files
files.download("MedIncome.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>